In [1]:

from tensorflow.keras import utils
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing import sequence
from baseline import my_model
import numpy as np
import pandas as pd

In [2]:
FEED_INFO = "./my_data/my_info_gene.csv"
I_HISTORY_DATA = "./my_data/train_14/train_history_i.csv"
DIS_HISTORY_DATA = "./my_data/train_14/train_history_dis.csv"
RECOM_DATA = "./my_data/train_14/train_recommend_2.csv"
RECOM_TARGET = "./my_data/train_14/train_target_2.csv"
GENE_FEED_EMB = "./general_feed_emd.npy"

# 因为每条推荐视频都要生成关于此用户的互动和不互动序列，太大，所以用生成器
feed_info = pd.read_csv(FEED_INFO, header=0, index_col=None).values
all_feed_id = feed_info[:, 0]
i_history = pd.read_csv(I_HISTORY_DATA, header=0, index_col=None).groupby(by=['userid'])
dis_history = pd.read_csv(DIS_HISTORY_DATA, header=0, index_col=None).groupby(by=['userid'])
recom_data = pd.read_csv(RECOM_DATA, header=0, index_col=None).values
recom_target = pd.read_csv(RECOM_TARGET, header=0, index_col=None).values

# test_quantity = int(len(recom_data) * 0.2)
# test_recom = recom_data[-test_quantity:]
# test_target = recom_target[-test_quantity:]

# train_recom = recom_data[:test_quantity]
# train_target = recom_target[:test_quantity]
train_recom = recom_data
train_target = recom_target

In [3]:
# 互动和不互动序列的填充长度按照最长的进行
i_maxlen = max(i_history.size().values)
dis_maxlen = max(dis_history.size().values)

# 新视频未知视频的填充数据，主要是为了提交阶段，其中feed_embedding用全部的均值来代替（我自己瞎想的）
new_feed_au_so_si_p = np.array([-2])
new_feed_tag = np.expand_dims(np.concatenate([np.ones(shape=(1, )), np.zeros(shape=(350, ))]), axis=0)
new_feed_emb = np.load("./general_feed_emd.npy")

# 假如某个用户互动或不互动序列为空时，填充
non_ausosip = -2

non_tag = np.concatenate([np.array([1]), np.zeros(shape=(350, ))])

non_i_action = np.zeros(shape=(4, ))

In [4]:
# csv读出来的东西不能直接用，需要转化一下
def convert_emb(x):
    return np.array(x.replace('[', '').replace(']', '').split(','), dtype=np.float)


def convert_tag(x):
    return np.array(x.replace('[', '').replace(']', '').replace('\n', '').split('. '), dtype=np.float)

In [5]:
# 生成器
def generator(the_data, the_target, batch_size=8):
    i = 0
    while True:

        if i + batch_size > the_data.shape[0]:
            data = the_data[i:recom_data.shape[0]]
            target = the_target[i:recom_data.shape[0]]
            i = 0
        else:
            data = the_data[i:i+batch_size]
            target = the_target[i:i+batch_size]
            i += batch_size

        i_author = []
        i_song = []
        i_singer = []
        i_psec = []
        i_tag = []
        i_emb = []
        i_action = []

        dis_author = []
        dis_song = []
        dis_singer = []
        dis_psec = []
        dis_tag = []
        dis_emb = []

        recom_author = []
        recom_song = []
        recom_singer = []
        recom_psec = []
        recom_tag = []
        recom_emb = []

        device = utils.to_categorical(data[:, -1] - 1, num_classes=2)

        # 对于batch_size中的用户依次生成数据
        # 用try是因为，可能有未知视频，或者互动、不互动序列为空
        for u in range(data.shape[0]):
            user = data[u][0]
            recom_feed = data[u][1]

            try:
                recom_info = feed_info[all_feed_id == recom_feed][0]
                recom_author.append([recom_info[1]])
                recom_psec.append([recom_info[2]])
                recom_singer.append([recom_info[3]])
                recom_song.append([recom_info[4]])
                recom_emb.append([convert_emb(recom_info[5])])
                recom_tag.append([convert_tag(recom_info[6])])
            except:
                recom_author.append([non_ausosip])
                recom_psec.append([non_ausosip])
                recom_singer.append([non_ausosip])
                recom_song.append([non_ausosip])
                recom_emb.append([new_feed_emb])
                recom_tag.append([non_tag])

            try:
                dis_feed = dis_history.get_group(user).values[:, 1]
                temp = feed_info[np.isin(all_feed_id, dis_feed, assume_unique=True)]
                dis_author.append(temp[:, 1])
                dis_psec.append(temp[:, 2])
                dis_singer.append(temp[:, 3])
                dis_song.append(temp[:, 4])
                dis_emb.append(list(map(convert_emb, temp[:, 5])))
                dis_tag.append(list(map(convert_tag, temp[:, 6])))
            except:
                dis_author.append([non_ausosip])
                dis_psec.append([non_ausosip])
                dis_singer.append([non_ausosip])
                dis_song.append([non_ausosip])
                dis_emb.append([new_feed_emb])
                dis_tag.append([non_tag])

            try:
                i_feed = i_history.get_group(user).values[:, 1]
                i_action.append(i_history.get_group(user).values[:, 2:])
                temp = feed_info[np.isin(all_feed_id, i_feed, assume_unique=True)]
                i_author.append(temp[:, 1])
                i_psec.append(temp[:, 2])
                i_singer.append(temp[:, 3])
                i_song.append(temp[:, 4])
                i_emb.append(list(map(convert_emb, temp[:, 5])))
                i_tag.append(list(map(convert_tag, temp[:, 6])))
            except:
                i_author.append([non_ausosip])
                i_psec.append([non_ausosip])
                i_singer.append([non_ausosip])
                i_song.append([non_ausosip])
                i_emb.append([new_feed_emb])
                i_tag.append([non_tag])
                i_action.append([non_i_action])

        # 最后将所有不长序列填充为定长序列，需要词典索引后emb的用-1，不需要的直接0
        yield [sequence.pad_sequences(i_author, value=-1, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_author, value=-1, padding='post', maxlen=dis_maxlen),
               np.array(recom_author),
               sequence.pad_sequences(i_song, value=-1, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_song, value=-1, padding='post', maxlen=dis_maxlen),
               np.array(recom_song),
               sequence.pad_sequences(i_singer, value=-1, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_singer, value=-1, padding='post', maxlen=dis_maxlen),
               np.array(recom_singer),
               sequence.pad_sequences(i_psec, value=-1, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_psec, value=-1, padding='post', maxlen=dis_maxlen),
               np.array(recom_psec),
               sequence.pad_sequences(i_tag, value=0, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_tag, value=0, padding='post', maxlen=dis_maxlen),
               np.array(recom_tag),
               sequence.pad_sequences(i_emb, value=0, padding='post', maxlen=i_maxlen),
               sequence.pad_sequences(dis_emb, value=0, padding='post', maxlen=dis_maxlen),
               np.array(recom_emb),
               sequence.pad_sequences(i_action, value=0, padding='post', maxlen=i_maxlen),
               device], target

In [11]:
# 批量大小
BATCH_SIZE = 32
test_BATCH_SIZE = 32

# 生成器
train_gene = generator(train_recom, train_target, batch_size=BATCH_SIZE)
# test_gene = generator(test_recom, test_target, batch_size=test_BATCH_SIZE)

In [ ]:

num = 0
for d in train_gene:

    num += 1
    print(num)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
# 模型
the_model = my_model.get_model(FEED_INFO)

Model: "My_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
i_author_input (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
i_song_input (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
i_singer (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
i_psec_input (InputLayer)       [(None, None)]       0                                            
___________________________________________________________________________________________

In [9]:
# 生成器参数
steps_per_epoch = int(train_recom.shape[0] / BATCH_SIZE)
# valid_steps = int(test_recom.shape[0] / test_BATCH_SIZE)

# 拟合
the_model.fit(train_gene,
              epochs=1,
              steps_per_epoch=steps_per_epoch)
              # validation_data=test_gene,
              # validation_steps=valid_steps,
              # validation_freq=1)

    4/25633 [..............................] - ETA: 210:42:18 - loss: 0.6499 - binary_accuracy: 0.7422 - auc: 0.4122

KeyboardInterrupt: 

In [2]:
import pandas as pd

f = pd.read_csv("./submit/bn_all_submit.csv", header=0, index_col=None).values
s = pd.read_csv("./submit/model_bnelu_cp_1_submit.csv", header=0, index_col=None).values
t = pd.read_csv("./submit/model_bnelu_3_submit.csv", header=0, index_col=None).values

In [3]:
fs = 0.616913
sc = 0.616951
tc = 0.6078

fss = fs /(fs + sc +tc)
scc = sc / (fs + sc + tc)
tcc = tc / (fs + sc + tc)

In [4]:
bag = fss * f[:, 2:] + scc * s[:, 2:] + tc * t[:, 2:]

In [5]:
import numpy as np
f_bag = np.concatenate([f[:, 0:2], bag], axis=1)

In [6]:
f_bag = pd.DataFrame(f_bag, columns=['userid', 'feedid', 'read_comment', 'like', 'click_avatar', 'forward'])
f_bag.to_csv("./submit/elu_bnall_bag_submit.csv")

In [3]:
import tensorflow as tf
from tensorflow import keras
mo = tf.keras.models.load_model("./baseline/model/try")
mo2 = tf.keras.models.load_model("./baseline/model/my_model_2")

In [5]:
print(mo.weights[10] == mo2.weights[10])

tf.Tensor(
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False Fal

In [2]:
import pandas as pd

score = pd.read_csv("./submit/model_2_again_submit.csv", index_col=None, header=0)
print(score)
score.to_csv('././submit/model_2_again_submit.csv', float_format='%.16f', index=None)

        userid  feedid  read_comment      like  click_avatar   forward
0        14298   67227      0.000292  0.025982      0.014012  0.000914
1        68356   91864      0.004389  0.008928      0.002446  0.000859
2        49925  104657      0.013430  0.000077      0.001140  0.000577
3        60529   23738      0.136601  0.000149      0.036747  0.000099
4       131482   69038      0.000017  0.002370      0.006186  0.000563
...        ...     ...           ...       ...           ...       ...
421980  133812   56450      0.009766  0.000177      0.000004  0.000039
421981  231669   76501      0.000003  0.059096      0.000452  0.000032
421982  179168   70550      0.008360  0.070097      0.000171  0.006843
421983   92546   49432      0.000726  0.001419      0.000101  0.005805
421984  205434    2109      0.000209  0.024861      0.000956  0.000009

[421985 rows x 6 columns]
